In [ ]:
!pip install neo4j 
!pip install pandas 
!pip install altair

     |████████████████████████████████| 75 kB 2.2 MB/s 
  Created wheel for neo4j: filename=neo4j-4.3.3-py3-none-any.whl size=99750 sha256=62eb14248d41bb085b102f5bcd76db4056beb0b4037d4a4f280a0f63baa386ba
  Stored in directory: /root/.cache/pip/wheels/f8/b8/97/ca430c50164dd35e74a6ec25556a1950e8c894b38d820c996b
Successfully built neo4j


In [ ]:
from getpass import getpass
user = getpass('Neo4j user: ')
password = getpass('Neo4j password: ')

Neo4j user: ··········
Neo4j password: ··········


In [ ]:
from neo4j import GraphDatabase
from collections import deque
import altair as alt
import pandas as pd

uri = r"bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
def query_against_neo4j(query, **kwargs):
    with driver.session(database='cameradeputati') as session:
        result = session.run(query, **kwargs)
        return deque(result)

In [ ]:
##################
###
# Deputati che hanno cambiato più partiti
##
##################

query = """
       MATCH (d:Persona)-[:ISCRITTO_AL_PARTITO]->(p:Lista) RETURN d.fullname as deputato, p.name as partito, 1 as tot 
        """

res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'partito': [r['partito'] for r in res],'tot': [r['tot'] for r in res]})




chart=alt.Chart(gnu_res).transform_joinaggregate(totale_partiti="sum(tot)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "partito:N",
      legend = alt.Legend(title = "Partito")
    ),
  tooltip=['deputato:N','partito:N','totale_partiti:Q']
)



chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno presentato più atti come primo firmatario, raggruppati per tipo di atto
##
##################

query = """
      MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
      WHERE r.ruolo = 'primo firmatario'
      RETURN p.fullname as deputato, a.tipo as tipo_atto, count(r) as tot ORDER BY tot DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'tipo_atto': [r['tipo_atto'] for r in res],'tot': [r['tot'] for r in res]})

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "tipo_atto:N",
      legend = alt.Legend(title = "Tipo Atto")
    ),
  tooltip=['deputato:N','tipo_atto:N','tot:Q','totale_deputato:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno presentato più atti anche come altro firmatario, raggruppati per tipo di atto
##
##################

query = """
      MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
      RETURN p.fullname as deputato, a.tipo as tipo_atto, count(r) as tot ORDER BY tot DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'tipo_atto': [r['tipo_atto'] for r in res],'tot': [r['tot'] for r in res]})

alt.data_transformers.enable('default', max_rows=None)

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "tipo_atto:N",
      legend = alt.Legend(title = "Tipo Atto")
    ),
  tooltip=['deputato:N','tipo_atto:N','tot:Q','totale_deputato:Q']
)
chart

alt.Chart(...)

In [ ]:
##################
###
# Partiti dei deputati che hanno presentato più atti come primo firmatario, raggruppati per tipo di atto
##
##################

query = """
    MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
    WHERE r.ruolo = 'primo firmatario'
    MATCH (p)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
    WHERE r2.start_date <= a.data <= r2.end_date 
    RETURN l.name as lista, a.tipo as tipo_atto, count(r) as tot
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'lista': [r['lista'] for r in res], 'tipo_atto': [r['tipo_atto'] for r in res],'tot': [r['tot'] for r in res]})

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_partito="sum(tot)", groupby=["lista"]
).mark_bar().encode(
  x=alt.X('lista:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "tipo_atto:N",
      legend = alt.Legend(title = "Tipo Atto")
    ),
  tooltip=['lista:N','tipo_atto:N','tot:Q','totale_partito:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Partiti dei deputati che hanno presentato più atti anche come altro firmatario, raggruppati per tipo di atto
##
##################

query = """
    MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
    MATCH (p)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
    WHERE r2.start_date <= a.data <= r2.end_date 
    RETURN l.name as lista, a.tipo as tipo_atto, count(r) as tot
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'lista': [r['lista'] for r in res], 'tipo_atto': [r['tipo_atto'] for r in res],'tot': [r['tot'] for r in res]})

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_partito="sum(tot)", groupby=["lista"]
).mark_bar().encode(
  x=alt.X('lista:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "tipo_atto:N",
      legend = alt.Legend(title = "Tipo Atto")
    ),
  tooltip=['lista:N','tipo_atto:N','tot:Q','totale_partito:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno presentato più Progetti di legge come primo firmatario
##
##################

query = """
      MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
      WHERE r.ruolo = 'primo firmatario'
      AND a.tipo = 'Progetto di Legge'
      RETURN p.fullname as deputato, count(r) as tot ORDER BY tot DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'tot': [r['tot'] for r in res]})

chart = alt.Chart(gnu_res).mark_bar().encode(
    x=alt.X('deputato:N',sort='-y'),
    y='tot:Q',
    tooltip=['deputato:N','tot:Q']
)
chart


alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno presentato più Progetti di legge anche come altro firmatario
##
##################

query = """
      MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
      WHERE a.tipo = 'Progetto di Legge'
      RETURN p.fullname as deputato, count(r) as tot ORDER BY tot DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'tot': [r['tot'] for r in res]})

chart = alt.Chart(gnu_res).mark_bar().encode(
    x=alt.X('deputato:N',sort='-y'),
    y='tot:Q',
    tooltip=['deputato:N','tot:Q']
)
chart


alt.Chart(...)

In [ ]:
##################
###
# Partiti dei deputati che hanno presentato più Progetti di legge come primo firmatario
##
##################

query = """
    MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
    WHERE r.ruolo = 'primo firmatario'
    AND a.tipo = 'Progetto di Legge'
    MATCH (p)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
    WHERE r2.start_date <= a.data <= r2.end_date 
    RETURN l.name as lista, count(r) as tot
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'lista': [r['lista'] for r in res], 'tot': [r['tot'] for r in res]})


chart = alt.Chart(gnu_res).mark_bar().encode(
    x=alt.X('lista:N',sort='-y'),
    y='tot:Q',
    tooltip=['lista:N','tot:Q']
)
chart

alt.Chart(...)

In [ ]:
##################
###
# Partiti dei deputati che hanno presentato più Progetti di legge anche come altro firmatario
##
##################

query = """
    MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)
    WHERE a.tipo = 'Progetto di Legge'
    MATCH (p)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
    WHERE r2.start_date <= a.data <= r2.end_date 
    RETURN l.name as lista, count(r) as tot
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'lista': [r['lista'] for r in res], 'tot': [r['tot'] for r in res]})


chart = alt.Chart(gnu_res).mark_bar().encode(
    x=alt.X('lista:N',sort='-y'),
    y='tot:Q',
    tooltip=['lista:N','tot:Q']
)
chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno presentato più emendamenti, raggruppati per partito
## TO DO PRRENDERE ANCHE GLI EMENDAMENTI ACCORPATI
##################

query = """
    MATCH (a:Atto)<-[:SI_RIFERISCE]-(n:Votazione) 
    WHERE n.tipoVotazione = 'Emendamento' 
    WITH distinct(n.descrizione) as descrizione 
    WITH apoc.text.regexGroups(descrizione,"(([A-Z\s]+) \([(A-Z)]+\) (e [0-9\\.\s]+))?([A-Z\s]+) \(([(A-Z5)]+)\)(\s+|,[a-zA-Z\s&#39;]+)?")[0] as rgx
    WHERE rgx is NOT NULL WITH trim(rgx[-2]) as partito, trim(rgx[-3]) as deputato 
    RETURN deputato, partito, count(*) as tot ORDER BY tot DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'partito': [r['partito'] for r in res],'tot': [r['tot'] for r in res]})
chart=alt.Chart(gnu_res).transform_joinaggregate(totale_partito="sum(tot)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "partito:N",
      legend = alt.Legend(title = "Tipo Atto")
    ),
  tooltip=['deputato:N','partito:N','tot:Q','totale_partito:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Partiti che hanno presentato più emendamenti, raggruppati per Deputato
##
##################

query = """
    MATCH (a:Atto)<-[:SI_RIFERISCE]-(n:Votazione) 
    WHERE n.tipoVotazione = 'Emendamento' 
    WITH distinct(n.descrizione) as descrizione 
    WITH apoc.text.regexGroups(descrizione,"(([A-Z\s]+) \([(A-Z)]+\) (e [0-9\\.\s]+))?([A-Z\s]+) \(([(A-Z5)]+)\)(\s+|,[a-zA-Z\s&#39;]+)?")[0] as rgx 
    WHERE rgx is NOT NULL 
    WITH trim(rgx[-2]) as partito, trim(rgx[-3]) as deputato 
    RETURN partito, deputato, count(*) as tot ORDER BY tot DESC        
      """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'partito': [r['partito'] for r in res],'tot': [r['tot'] for r in res]})
chart=alt.Chart(gnu_res).transform_joinaggregate(totale_partito="sum(tot)", groupby=["partito"]
).mark_bar().encode(
  x=alt.X('partito:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "deputato:N",
      legend = alt.Legend(title = "Tipo Atto")
    ),
  tooltip=['partito:N','deputato:N','tot:Q','totale_partito:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Atti che hanno ricevuto più emendamenti, raggruppati per partito di provenienza
## TO DO: AGGIUSTARE URL!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Per alcuni emendamenti in quanto scritti a mano e con sporcizia non è possibile estrarre il partito
##
##################


query = """
      MATCH (a:Atto)<-[:SI_RIFERISCE]-(n:Votazione) 
      WHERE n.tipoVotazione = 'Emendamento' 
      WITH a.uri as atto, trim(apoc.text.regexGroups(n.descrizione,"(([A-Z\s]+) \([(A-Z)]+\) (e [0-9\\.\s]+))?([A-Z\s]+) \(([(A-Z5)]+)\)(\s+|,[a-zA-Z\s&#39;]+)?")[0][-2]) as partito
      WHERE partito IS NOT NULL 
      RETURN atto, partito, count(*) as tot_emendamenti    
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'atto': [r['atto'] for r in res], 'partito': [r['partito'] for r in res], 'tot_emendamenti': [r['tot_emendamenti'] for r in res] })

chart=alt.Chart(gnu_res).configure_bar(href='atto').transform_joinaggregate(totale_atto="sum(tot_emendamenti)", groupby=["atto"]
).mark_bar().encode(
  x=alt.X('atto:N',sort='-y'),
  y='tot_emendamenti:Q',    
  color = alt.Color(
      "partito:N",
      legend = alt.Legend(title = "Partito")
    ),
  tooltip=['atto:N','partito:N','tot_emendamenti:Q','totale_atto:Q']
)

chart


alt.Chart(...)

In [ ]:
##################
###
# Deputati i cui atti presentati come primo firmatario hanno ricevuto più emendamenti
## 
##
##################


query = """
      MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)<-[:SI_RIFERISCE]-(n:Votazione) 
      WHERE n.tipoVotazione = 'Emendamento' 
      AND r.ruolo = 'primo firmatario'
      WITH  p.fullname as deputato, a.uri as atto ,collect(distinct n.descrizione) as emendamenti
      RETURN deputato,atto, size(emendamenti) as tot_emendamenti
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'atto': [r['atto'] for r in res], 'tot_emendamenti': [r['tot_emendamenti'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot_emendamenti)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot_emendamenti:Q',    
  color = alt.Color(
      "atto:N",
      legend = alt.Legend(title = "Atto")
    ),
  tooltip=['deputato:N','atto:N','tot_emendamenti:Q', 'totale_deputato:Q']
)

chart


alt.Chart(...)

In [ ]:
##################
###
# Deputati i cui atti presentati anche come altro firmatario hanno ricevuto più emendamenti
## 
##
##################


query = """
      MATCH (p:Persona)-[r:PRESENTA]->(a:Atto)<-[:SI_RIFERISCE]-(n:Votazione) 
      WHERE n.tipoVotazione = 'Emendamento' 
      WITH  p.fullname as deputato, a.uri as atto ,collect(distinct n.descrizione) as emendamenti
      RETURN deputato,atto, size(emendamenti) as tot_emendamenti
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'atto': [r['atto'] for r in res], 'tot_emendamenti': [r['tot_emendamenti'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot_emendamenti)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot_emendamenti:Q',    
  color = alt.Color(
      "atto:N",
      legend = alt.Legend(title = "Atto")
    ),
  tooltip=['deputato:N','atto:N','tot_emendamenti:Q', 'totale_deputato:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno votato un maggior numero di volte favorevole ad Atti poi approvati, raggruppati per maggioranza o opposizione
## 
##
##################


query = """
      MATCH (d:Persona)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione:Approvato)
      WHERE voto.espressione = 'Favorevole'
      WITH d.fullname as fullname, v.uri as approvato, voto
        WITH
        CASE  [lab in labels(voto) WHERE lab STARTS WITH 'ProvenienteDa' ][0]
        WHEN 'ProvenienteDaMaggioranza' THEN 'maggioranza'
        WHEN 'ProvenienteDaOpposizione' THEN 'opposizione'
        WHEN 'precedenteAlPrimoGovernoConte' THEN 'precedente_al_conte_primo'
        END as provenienza, fullname, approvato
      WHERE provenienza is not null
      WITH fullname, provenienza, collect(approvato) as approvati
      RETURN fullname, provenienza, size(approvati) as tot_approvati ORDER BY size(approvati) DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['fullname'] for r in res], 'provenienza': [r['provenienza'] for r in res], 'tot_approvati': [r['tot_approvati'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot_approvati)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot_approvati:Q',    
  color = alt.Color(
      "provenienza:N",
      legend = alt.Legend(title = "Provenienza")
    ),
  tooltip=['deputato:N','provenienza:N','tot_approvati:Q','totale_deputato:Q']
)
chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che hanno votato un maggior numero di volte favorevole ad Atti legislativi poi approvati, raggruppati per maggioranza o opposizione
## 
##
##################


query = """
      MATCH (d:Persona)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione:Approvato)-[:SI_RIFERISCE]->(a:Atto)
      WHERE voto.espressione = 'Favorevole'
      AND [x in labels(a) WHERE x in ['Parlamentare','Regioni','CNEL','Governo','Popolare'] ]
      WITH d.fullname as fullname, v.uri as approvato, voto
        WITH
        CASE  [lab in labels(voto) WHERE lab STARTS WITH 'ProvenienteDa' ][0]
        WHEN 'ProvenienteDaMaggioranza' THEN 'maggioranza'
        WHEN 'ProvenienteDaOpposizione' THEN 'opposizione'
        WHEN 'precedenteAlPrimoGovernoConte' THEN 'precedente_al_conte_primo'
        END as provenienza, fullname, approvato
      WHERE provenienza is not null
      WITH fullname, provenienza, collect(approvato) as approvati
      RETURN fullname, provenienza, size(approvati) as tot_approvati ORDER BY size(approvati) DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['fullname'] for r in res], 'provenienza': [r['provenienza'] for r in res], 'tot_approvati': [r['tot_approvati'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot_approvati)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot_approvati:Q',    
  color = alt.Color(
      "provenienza:N",
      legend = alt.Legend(title = "Provenienza")
    ),
  tooltip=['deputato:N','provenienza:N','tot_approvati:Q','totale_deputato:Q']
)
chart

alt.Chart(...)

In [ ]:


##################
###
# Deputati dell'opposizione che hanno votato un maggior numero di volte favorevole ad Atti poi approvati, raggruppati per partito
## 
##
##################


query = """
      MATCH (d:Persona)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione:Approvato)
      WHERE voto:Favorevole:ProvenienteDaOpposizione
      MATCH (d)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
      WHERE r2.start_date <= v.data <= r2.end_date 
      WITH d.fullname as deputato, l.name as partito, collect(v) as approvati
      RETURN deputato, partito, size(approvati) as tot_approvati ORDER BY size(approvati) DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'partito': [r['partito'] for r in res], 'tot_approvati': [r['tot_approvati'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot_approvati)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot_approvati:Q',    
  color = alt.Color(
      "partito:N",
      legend = alt.Legend(title = "Partito")
    ),
  tooltip=['deputato:N','partito:N','tot_approvati:Q', 'totale_deputato:Q']
)
chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati dell'opposizione che hanno votato un maggior numero di volte favorevole ad Atti legislativi poi approvati, raggruppati per partito
## 
##
##################


query = """
      MATCH (d:Persona)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione:Approvato)-[:SI_RIFERISCE]->(a:Atto)
      WHERE voto:Favorevole:ProvenienteDaOpposizione
      AND [x in labels(a) WHERE x in ['Parlamentare','Regioni','CNEL','Governo','Popolare'] ]
      MATCH (d)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
      WHERE r2.start_date <= v.data <= r2.end_date 
      WITH d.fullname as deputato, l.name as partito, collect(v) as approvati
      RETURN deputato, partito, size(approvati) as tot_approvati ORDER BY size(approvati) DESC
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'partito': [r['partito'] for r in res], 'tot_approvati': [r['tot_approvati'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot_approvati)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot_approvati:Q',    
  color = alt.Color(
      "partito:N",
      legend = alt.Legend(title = "Partito")
    ),
  tooltip=['deputato:N','partito:N','tot_approvati:Q', 'totale_deputato:Q']
)
chart

alt.Chart(...)

In [ ]:


##################
###
# Emendamenti per deputati,raggruppati per maggioranza o opposizione
## 
##
##################


query = """
      MATCH (a:Atto)<-[:SI_RIFERISCE]-(n:Votazione) 
      WHERE n.tipoVotazione = 'Emendamento'
      AND NOT a:ProvenienteDaMaggioranza:ProvenienteDaOpposizione 
      WITH distinct(n.descrizione) as descrizione, n 
      WITH apoc.text.regexGroups(descrizione,"(([A-Z\s]+) \([(A-Z)]+\) (e [0-9\\.\s]+))?([A-Z\s]+) \(([(A-Z5)]+)\)(\s+|,[a-zA-Z\s&#39;]+)?")[0] as rgx, n 
      WHERE rgx is NOT NULL 
      WITH trim(rgx[-2]) as partito, trim(rgx[-3]) as deputato, n
      MATCH (p:Persona)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(n)
      WHERE p.cognome + ' ' + p.nome = deputato
      WITH deputato, partito, voto, n
        WITH
        CASE  [lab in labels(voto) WHERE lab STARTS WITH 'ProvenienteDa' ][0]
        WHEN 'ProvenienteDaMaggioranza' THEN 'maggioranza'
        WHEN 'ProvenienteDaOpposizione' THEN 'opposizione'
        WHEN 'precedenteAlPrimoGovernoConte' THEN 'precedente_al_conte_primo'
        END as provenienza, deputato, partito, n
      WHERE provenienza is not null
      RETURN deputato, provenienza, count(n) as tot
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res], 'provenienza': [r['provenienza'] for r in res], 'tot': [r['tot'] for r in res] })

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "provenienza:N",
      legend = alt.Legend(title = "Provenienza")
    ),
  tooltip=['deputato:N','provenienza:N','tot:Q', 'totale_deputato:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Deputati che non hanno votato a un maggior numero di votazioni - non per astensione ma per assenza, raggruppati per partito
##
##################

query = """
    MATCH (p:Persona)-[r:HA_VOTATO]->(voto:Voto:`Non ha votato`)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione)
    MATCH (p)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
    WHERE r2.start_date <= v.data <= r2.end_date 
    RETURN p.fullname as deputato, l.name as lista, count(v) as tot
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'lista': [r['lista'] for r in res], 'deputato': [r['deputato'] for r in res],'tot': [r['tot'] for r in res]})

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_deputato="sum(tot)", groupby=["deputato"]
).mark_bar().encode(
  x=alt.X('deputato:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "lista:N",
      legend = alt.Legend(title = "Partito")
    ),
  tooltip=['deputato:N','lista:N','tot:Q','totale_deputato:Q']
)

chart

alt.Chart(...)

In [ ]:
##################
###
# Partiti dei deputati che non hanno votato a un maggior numero di votazioni - non per astensione ma per assenza, raggruppati per deputato
##
##################

query = """
    MATCH (p:Persona)-[r:HA_VOTATO]->(voto:Voto:`Non ha votato`)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione)
    MATCH (p)-[r2:ISCRITTO_AL_PARTITO]->(l:Lista)
    WHERE r2.start_date <= v.data <= r2.end_date 
    RETURN p.fullname as deputato, l.name as lista, count(v) as tot

        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'lista': [r['lista'] for r in res], 'deputato': [r['deputato'] for r in res],'tot': [r['tot'] for r in res]})

chart=alt.Chart(gnu_res).transform_joinaggregate(totale_partito="sum(tot)", groupby=["lista"]
).mark_bar().encode(
  x=alt.X('lista:N',sort='-y'),
  y='tot:Q',    
  color = alt.Color(
      "deputato:N",
      legend = alt.Legend(title = "Deputato")
    ),
  tooltip=['lista:N','deputato:N','tot:Q','totale_partito:Q']
)

chart

alt.Chart(...)

In [ ]:

##################
###
# Atti della camera suddivisi per iniziativa
##
##################

query = """
  MATCH (a:Atto)
  WHERE NOT a:AttodiControllo
  AND NOT a:AttodiIndirizzo
  WITH labels(a) as labes
          WITH
          CASE  [lab in labes WHERE lab in ['Parlamentare','Regioni','CNEL','Governo','Popolare'] ][0]
          WHEN 'Parlamentare' THEN 'parlamentare'
          WHEN 'Regioni' THEN 'regioni'
          WHEN 'CNEL' THEN 'cnel'
          WHEN 'Governo' THEN 'governo'
          WHEN 'Popolare' THEN 'popolare'
          END as iniziativa, labes
        WHERE iniziativa IS NOT null
        RETURN iniziativa, count(labes) as tot
    
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'iniziativa': [r['iniziativa'] for r in res], 'tot': [r['tot'] for r in res]})


chart = alt.Chart(gnu_res).mark_bar().encode(
    x=alt.X('iniziativa:N',sort='-y'),
    y='tot:Q',
    tooltip=['iniziativa:N','tot:Q']
)
chart

alt.Chart(...)

In [ ]:

##################
###
# Numero di Atti suddivisi per governo
##
##################

query = """
    MATCH (a:Atto)
    WHERE [x in labels(a) WHERE NOT x in ['AttodiControllo','AttodiIndirizzo', 'precedenteAlPrimoGovernoConte' ]]
    WITH collect(a) as atti
    RETURN size([x in atti WHERE x.data<= date('2019-09-04')]) as primo_conte,
          size([x in atti WHERE date('2019-09-05') <= x.data <= date('2021-02-12')]) as secondo_conte,
          size([x in atti WHERE date('2021-02-13') <= x.data]) as draghi
    
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'primo_conte': [r['primo_conte'] for r in res], 'secondo_conte': [r['secondo_conte'] for r in res], 'draghi': [r['draghi'] for r in res]})


gnu_res

,primo_conte,secondo_conte,draghi
0,13323,15484,4690


In [ ]:
##################
###
# Numero di Atti approvati suddivisi per governo
##
##################

query = """
      MATCH (a:Atto)<-[:SI_RIFERISCE]-(v:Votazione)
      WHERE v.approvato = true 
      AND v.votazioneFinale = true
      AND [x in labels(a) WHERE NOT x in ['AttodiControllo','AttodiIndirizzo', 'precedenteAlPrimoGovernoConte' ]]
      WITH collect(a) as atti
      RETURN size([x in atti WHERE x.data<= date('2019-09-04')]) as primo_conte, 
            size([x in atti WHERE date('2019-09-05') <= x.data <= date('2021-02-12')]) as secondo_conte,  
            size([x in atti WHERE date('2021-02-13') <= x.data]) as draghi
    
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'primo_conte': [r['primo_conte'] for r in res], 'secondo_conte': [r['secondo_conte'] for r in res], 'draghi': [r['draghi'] for r in res]})


gnu_res

,primo_conte,secondo_conte,draghi
0,135,82,12


In [ ]:
##################
###
# Numero di Atti approvati suddivisi per iniziativa
##
##################

query = """
    MATCH (a:Atto)<-[:SI_RIFERISCE]-(v:Votazione)
    WHERE v.approvato = true 
    AND v.votazioneFinale = true
    AND [x in labels(a) WHERE NOT x in ['AttodiControllo','AttodiIndirizzo', 'precedenteAlPrimoGovernoConte' ]] 
    WITH labels(a) as labes
            WITH
              CASE  [lab in labes WHERE lab in ['Parlamentare','Regioni','CNEL','Governo','Popolare'] ][0]
              WHEN 'Parlamentare' THEN 'parlamentare'
              WHEN 'Regioni' THEN 'regioni'
              WHEN 'CNEL' THEN 'cnel'
              WHEN 'Governo' THEN 'governo'
              WHEN 'Popolare' THEN 'popolare'
              END as iniziativa, labes
    WHERE iniziativa IS NOT null
    RETURN iniziativa, count(labes) as tot
    
        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'iniziativa': [r['iniziativa'] for r in res], 'tot': [r['tot'] for r in res]})

chart = alt.Chart(gnu_res).mark_bar().encode(
    x=alt.X('iniziativa:N',sort='-y'),
    y='tot:Q',
    tooltip=['iniziativa:N','tot:Q']
)
chart

alt.Chart(...)

In [ ]:
##################
###
# Numero di Atti approvati suddivisi per governo e per deputato
##
##
##################

query = """
    MATCH (a:Atto)<-[:SI_RIFERISCE]-(v:Votazione)
    WHERE v.approvato = true 
    AND v.votazioneFinale = true
    AND [x in labels(a) WHERE NOT x in ['AttodiControllo','AttodiIndirizzo', 'precedenteAlPrimoGovernoConte' ]]
    WITH a as atto , v.data as data
    MATCH (p:Persona)-[r:PRESENTA]->(atto)
    WHERE r.ruolo = 'primo firmatario'
    WITH p.fullname as deputato, collect(atto) as atti
    RETURN deputato,  size([x in atti WHERE x.data<= date('2019-09-04')]) as primo_conte, 
          size([x in atti WHERE date('2019-09-05') <= x.data <= date('2021-02-12')]) as secondo_conte,  
          size([x in atti WHERE date('2021-02-13') <= x.data]) as draghi

        """
res = query_against_neo4j(query=query)

gnu_res = pd.DataFrame({'deputato': [r['deputato'] for r in res],'primo_conte': [r['primo_conte'] for r in res], 'secondo_conte': [r['secondo_conte'] for r in res], 'draghi' : [r['draghi'] for r in res]})

gnu_res

,deputato,primo_conte,secondo_conte,draghi
0,IOLANDA DI STASIO,0,1,0
1,MARZIO LIUNI,1,0,0
2,LAURA BOLDRINI,1,1,0
3,GIORGIA ANDREUZZA,0,1,0
4,GIUSI BARTOLOZZI,0,1,0
5,EMANUELE FIANO,1,0,0
6,GIORGIO MULE',0,1,0
7,EMANUELA CORDA,1,0,0
8,GRAZIANO DELRIO,1,0,0
9,MARIA CRISTINA CARETTA,0,1,0


In [ ]:
##################
###
# Similarità (indice di jaccard) tra deputati in base ai voti espressi e alle assenze
##
##################

query_cypher_projection = """
    CALL gds.graph.create.cypher(
        'votazioniPerDeputato',
        'MATCH (n) WHERE n:Persona OR n:Votazione RETURN id(n) AS id, labels(n) AS labels',
        'MATCH (p:Persona)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione)
        RETURN id(p) AS source, id(v) AS target, voto.espressione AS espressione'
    )
    
        """
res = query_against_neo4j(query=query)

In [ ]:
query = """
    MATCH (p:Persona {fullname:'GALEAZZO BIGNAMI'})-[:PRESENTA]->(a:Atto)
    WHERE a.tipo in ['INTERROGAZIONE A RISPOSTA SCRITTA']
    WITH a.uri as uri
    CALL apoc.load.html(uri,{div:".c2 > div"}) yield value
    WITH value.div[0].text as testoAtto
    WITH  apoc.text.join(split(testoAtto, '.')[3..], ',') as text2extract
    CALL apoc.load.jsonParams(
      "https://api.dbpedia-spotlight.org/it/annotate?text=" + text2extract + "&confidence=0.8&support=20",
      {accept:"application/json"},
      null
      ) YIELD value
      WITH value.Resources as resources
      WITH [elem in resources WHERE elem.`@surfaceForm` IS NOT NULL | replace(split(elem.`@surfaceForm`, "/")[-1],"_"," ") ] as uris
      RETURN apoc.coll.toSet(uris) as entities LIMIT 50
    """

res = query_against_neo4j(query=query)
[r['entities'] for r in res]

[['Massimo', 'Dario Franceschini'],
 ['Istituto nazionale previdenza sociale', 'Italia', 'Inps'],
 None,
 ['emergenza sanitaria',
  'COVID-19',
  'Inps',
  'ente previdenziale',
  'posta raccomandata'],
 ['Bologna', 'Budrio'],
 ['Fratelli Musulmani',
  'islam',
  'intelligence',
  'Hamas',
  'India',
  'Pakistan',
  'Boko Haram',
  'Africa',
  'Al-Qaeda',
  'Bahrain',
  'Egitto',
  'Arabia Saudita',
  'Emirati Arabi Uniti',
  'amministrazione Trump',
  'Fratellanza musulmana',
  'Washington',
  'Italia',
  'fratellanza musulmana',
  'Imam',
  'Grande Moschea',
  'Roma',
  '1974'],
 ['isole Eolie',
  'emergenza sanitaria',
  'Lipari',
  'prezioso',
  'Milazzo',
  'Messina',
  'Calabria',
  'elisoccorso',
  'orografia',
  'climatologia',
  '2011'],
 ['polmonite',
  'Cina',
  'Giuseppe',
  'Spallanzani',
  '2006',
  'Italia',
  'Consiglio dei ministri',
  'pandemia',
  'dia'],
 ['gasolio', 'diesel', 'energia termica', 'Italia', '2011', 'soia'],
 ['tiro a segno', 'sport', '1978', 'Roma', '

In [ ]:
query_persone = """
      MATCH (p:Persona)
      RETURN p.fullname as fullname
              """
res = query_against_neo4j(query=query_persone)
print(res)
for elem in res:
    print('elem'+str(elem))
    query = f"""
        MATCH (p:Persona {{fullname: "{elem['fullname']}"}})
        MATCH (p)-[:HA_VOTATO]->(voto:Voto)-[:RELATIVO_ALLA_VOTAZIONE]->(v:Votazione)
        RETURN p.fullname as fullname,v.denominazione as denominazione, voto.espressione as espressione
            """
    print(query)
    results = query_against_neo4j(query=query)
    print(results)
    deputati_espressioni = {}
    for elem in results:
      print('gnu'+str(elem))
      deputati_espressioni[elem['fullname']] = {elem['denominazione'] : elem['espressione']}

deputati_espressioni